In [1]:
import zmq
import sys
import threading
import time
from random import randint, random

__author__ = "Felipe Cruz <felipecruz@loogica.net>"
__license__ = "MIT/X11"

def tprint(msg):
    """like print, but won't get newlines confused with multiple threads"""
    sys.stdout.write(msg + '\n')
    sys.stdout.flush()

class ClientTask(threading.Thread):
    """ClientTask"""
    def __init__(self, id):
        self.id = id
        threading.Thread.__init__ (self)

    def run(self):
        context = zmq.Context()
        socket = context.socket(zmq.DEALER)
        identity = u'worker-%d' % self.id
        socket.identity = identity.encode('ascii')
        socket.connect('tcp://localhost:5570')
        print('Client %s started' % (identity))
        poll = zmq.Poller()
        poll.register(socket, zmq.POLLIN)
        reqs = 0
        while True:
            reqs = reqs + 1
            print('Req #%d sent..' % (reqs))
            socket.send_string(u'request #%d' % (reqs))
            for i in range(5):
                sockets = dict(poll.poll(1000))
                if socket in sockets:
                    msg = socket.recv()
                    tprint('Client %s received: %s' % (identity, msg))

        socket.close()
        context.term()

class ServerTask(threading.Thread):
    """ServerTask"""
    def __init__(self):
        threading.Thread.__init__ (self)

    def run(self):
        context = zmq.Context()
        frontend = context.socket(zmq.ROUTER)
        frontend.bind('tcp://*:5570')

        backend = context.socket(zmq.DEALER)
        backend.bind('inproc://backend')

        workers = []
        for i in range(5):
            worker = ServerWorker(context)
            worker.start()
            workers.append(worker)

        zmq.proxy(frontend, backend)

        frontend.close()
        backend.close()
        context.term()

class ServerWorker(threading.Thread):
    """ServerWorker"""
    def __init__(self, context):
        threading.Thread.__init__ (self)
        self.context = context

    def run(self):
        worker = self.context.socket(zmq.DEALER)
        worker.connect('inproc://backend')
        tprint('Worker started')
        while True:
            ident, msg = worker.recv_multipart()
            tprint('Worker received %s from %s' % (msg, ident))
            replies = randint(0,4)
            for i in range(replies):
                time.sleep(1. / (randint(1,10)))
                worker.send_multipart([ident, msg])

        worker.close()

def main():
    """main function"""
    server = ServerTask()
    server.start()
    for i in range(3):
        client = ClientTask(i)
        client.start()

    server.join()

if __name__ == "__main__":
    main()

Worker started
Client worker-0 startedWorker started
Client worker-1 startedWorker started
Client worker-2 started


Worker started
Worker started
Req #1 sent..Req #1 sent..Req #1 sent..Worker received b'request #1' from b'worker-0'

Worker received b'request #1' from b'worker-1'

Worker received b'request #1' from b'worker-2'

Client worker-2 received: b'request #1'
Client worker-1 received: b'request #1'
Client worker-0 received: b'request #1'
Req #2 sent..
Worker received b'request #2' from b'worker-0'
Req #2 sent..
Worker received b'request #2' from b'worker-2'
Req #2 sent..
Worker received b'request #2' from b'worker-1'
Client worker-0 received: b'request #2'
Client worker-2 received: b'request #2'
Client worker-1 received: b'request #2'
Client worker-0 received: b'request #2'
Client worker-0 received: b'request #2'
Client worker-0 received: b'request #2'
Req #3 sent..
Worker received b'request #3' from b'worker-0'
Client worker-0 received: b'request #3'
Client worker-0 received: 

KeyboardInterrupt: 

Client worker-0 received: b'request #4'
Client worker-0 received: b'request #4'
Req #4 sent..
Worker received b'request #4' from b'worker-2'
Req #4 sent..
Worker received b'request #4' from b'worker-1'
Client worker-2 received: b'request #4'
Client worker-2 received: b'request #4'
Client worker-1 received: b'request #4'
Client worker-2 received: b'request #4'
Client worker-1 received: b'request #4'
Client worker-1 received: b'request #4'
Client worker-2 received: b'request #4'
Client worker-1 received: b'request #4'
Req #5 sent..
Worker received b'request #5' from b'worker-0'
Req #5 sent..
Worker received b'request #5' from b'worker-2'
Req #5 sent..
Worker received b'request #5' from b'worker-1'
Client worker-1 received: b'request #5'
Req #6 sent..
Worker received b'request #6' from b'worker-1'
Client worker-1 received: b'request #6'
Req #6 sent..
Worker received b'request #6' from b'worker-0'
Client worker-0 received: b'request #6'
Client worker-1 received: b'request #6'
Req #6 sent..

Client worker-2 received: b'request #23'
Req #22 sent..
Client worker-2 received: b'request #23'
Worker received b'request #22' from b'worker-1'
Client worker-2 received: b'request #23'
Client worker-1 received: b'request #22'
Client worker-1 received: b'request #22'
Req #20 sent..
Worker received b'request #20' from b'worker-0'
Req #24 sent..
Worker received b'request #24' from b'worker-2'
Client worker-2 received: b'request #24'
Client worker-2 received: b'request #24'
Req #23 sent..
Worker received b'request #23' from b'worker-1'
Client worker-1 received: b'request #23'
Client worker-1 received: b'request #23'
Client worker-1 received: b'request #23'
Client worker-1 received: b'request #23'
Req #25 sent..
Worker received b'request #25' from b'worker-2'
Req #24 sent..
Worker received b'request #24' from b'worker-1'
Req #21 sent..
Worker received b'request #21' from b'worker-0'
Client worker-2 received: b'request #25'
Client worker-1 received: b'request #24'
Client worker-2 received: 

Client worker-0 received: b'request #37'
Client worker-0 received: b'request #37'
Client worker-0 received: b'request #37'
Req #42 sent..
Worker received b'request #42' from b'worker-2'
Client worker-2 received: b'request #42'
Req #43 sent..
Worker received b'request #43' from b'worker-1'
Req #38 sent..
Worker received b'request #38' from b'worker-0'
Client worker-0 received: b'request #38'
Client worker-1 received: b'request #43'
Client worker-0 received: b'request #38'
Client worker-1 received: b'request #43'
Req #43 sent..
Worker received b'request #43' from b'worker-2'
Req #39 sent..
Worker received b'request #39' from b'worker-0'
Req #44 sent..
Worker received b'request #44' from b'worker-1'
Client worker-0 received: b'request #39'
Client worker-1 received: b'request #44'
Client worker-0 received: b'request #39'
Client worker-1 received: b'request #44'
Client worker-0 received: b'request #39'
Client worker-1 received: b'request #44'
Req #40 sent..
Worker received b'request #40' fr

...

Req #4559 sent..
Worker received b'request #4559' from b'worker-1'
Client worker-0 received: b'request #4532'
Client worker-1 received: b'request #4559'
Client worker-0 received: b'request #4532'
Client worker-0 received: b'request #4532'
Client worker-0 received: b'request #4532'
Req #4533 sent..
Worker received b'request #4533' from b'worker-0'
Req #4535 sent..
Worker received b'request #4535' from b'worker-2'
Client worker-2 received: b'request #4535'
Client worker-2 received: b'request #4535'
Req #4560 sent..
Worker received b'request #4560' from b'worker-1'
Client worker-1 received: b'request #4560'
Client worker-1 received: b'request #4560'
Client worker-1 received: b'request #4560'
Req #4536 sent..
Worker received b'request #4536' from b'worker-2'
Req #4561 sent..
Worker received b'request #4561' from b'worker-1'
Req #4534 sent..
Worker received b'request #4534' from b'worker-0'
Req #4537 sent..
Worker received b'request #4537' from b'worker-2'
Client worker-2 received: b'reques

Client worker-1 received: b'request #4577'
Client worker-0 received: b'request #4549'
Client worker-0 received: b'request #4549'
Req #4554 sent..
Worker received b'request #4554' from b'worker-2'
Client worker-1 received: b'request #4577'
Req #4550 sent..
Worker received b'request #4550' from b'worker-0'
Client worker-2 received: b'request #4554'
Client worker-2 received: b'request #4554'
Client worker-2 received: b'request #4554'
Client worker-2 received: b'request #4554'
Req #4555 sent..
Worker received b'request #4555' from b'worker-2'
Client worker-2 received: b'request #4555'
Client worker-2 received: b'request #4555'
Req #4578 sent..
Worker received b'request #4578' from b'worker-1'
Client worker-1 received: b'request #4578'
Client worker-1 received: b'request #4578'
Client worker-1 received: b'request #4578'
Req #4556 sent..
Worker received b'request #4556' from b'worker-2'
Client worker-2 received: b'request #4556'
Req #4579 sent..
Worker received b'request #4579' from b'worker

Req #4595 sent..
Worker received b'request #4595' from b'worker-1'
Client worker-1 received: b'request #4595'
Client worker-0 received: b'request #4566'
Client worker-1 received: b'request #4595'
Client worker-1 received: b'request #4595'
Client worker-1 received: b'request #4595'
Req #4596 sent..
Worker received b'request #4596' from b'worker-1'
Client worker-1 received: b'request #4596'
Req #4573 sent..
Worker received b'request #4573' from b'worker-2'
Client worker-1 received: b'request #4596'
Client worker-1 received: b'request #4596'
Client worker-2 received: b'request #4573'
Client worker-1 received: b'request #4596'
Client worker-2 received: b'request #4573'
Req #4597 sent..
Worker received b'request #4597' from b'worker-1'
Client worker-1 received: b'request #4597'
Client worker-1 received: b'request #4597'
Req #4567 sent..
Worker received b'request #4567' from b'worker-0'
Client worker-0 received: b'request #4567'
Req #4574 sent..
Worker received b'request #4574' from b'worker

Client worker-2 received: b'request #4589'
Req #4584 sent..
Worker received b'request #4584' from b'worker-0'
Client worker-0 received: b'request #4584'
Req #4612 sent..
Worker received b'request #4612' from b'worker-1'
Client worker-1 received: b'request #4612'
Req #4590 sent..
Worker received b'request #4590' from b'worker-2'
Client worker-2 received: b'request #4590'
Client worker-2 received: b'request #4590'
Client worker-2 received: b'request #4590'
Client worker-2 received: b'request #4590'
Req #4591 sent..
Worker received b'request #4591' from b'worker-2'
Req #4585 sent..
Worker received b'request #4585' from b'worker-0'
Req #4613 sent..
Worker received b'request #4613' from b'worker-1'
Req #4592 sent..
Worker received b'request #4592' from b'worker-2'
Client worker-2 received: b'request #4592'
Req #4586 sent..
Worker received b'request #4586' from b'worker-0'
Client worker-2 received: b'request #4592'
Client worker-0 received: b'request #4586'
Client worker-0 received: b'reques

Client worker-2 received: b'request #4608'
Client worker-2 received: b'request #4608'
Req #4609 sent..
Worker received b'request #4609' from b'worker-2'
Client worker-2 received: b'request #4609'
Client worker-2 received: b'request #4609'
Req #4631 sent..
Worker received b'request #4631' from b'worker-1'
Client worker-2 received: b'request #4609'
Client worker-1 received: b'request #4631'
Req #4600 sent..
Worker received b'request #4600' from b'worker-0'
Client worker-0 received: b'request #4600'
Client worker-1 received: b'request #4631'
Client worker-1 received: b'request #4631'
Req #4610 sent..
Worker received b'request #4610' from b'worker-2'
Client worker-2 received: b'request #4610'
Req #4632 sent..
Worker received b'request #4632' from b'worker-1'
Client worker-1 received: b'request #4632'
Client worker-1 received: b'request #4632'
Client worker-1 received: b'request #4632'
Req #4601 sent..
Worker received b'request #4601' from b'worker-0'
Req #4633 sent..
Worker received b'requ